In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import fiona
import pyproj
import rtree
import shapely

import pandas as pd
import geopandas as gpd

import geotext

from urllib import request
from geotext import GeoText

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

from shapely.geometry import Point, Polygon
import descartes

import geograpy
import nltk

In [36]:
top_songs = pd.read_csv('top_songs_lyrics.csv', index_col=0)

In [37]:
top_songs.head()

,Rank,Artist,Song,Year,Lyrics
0,1,Glenn Miller,Chattanooga Choo Choo,1941,"Chattanooga Choo Choo Pardon me, boy Is that t..."
1,2,Sammy Kaye,Daddy,1941,Daddy Ladada ladada da dada Ladada ladada da d...
2,3,Artie Shaw,Stardust,1941,Stardust And now the purple dusk of twilight t...
3,4,The Andrews Sisters,Boogie Woogie Bugle Boy,1941,Boogie Woogie Bugle Boy He was a famous trumpe...
4,5,Jimmy Dorsey,Amapola (Pretty Little Poppy),1941,words


In [11]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [20]:
test_lyric = top_songs.at[0,'Lyrics']
doc=nlp(test_lyric)

C:\Users\josep\Anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [21]:
for ent in doc.ents:
    print(ent.text, ent.label_, ent.start, ent.end)

Chattanooga GPE 10 11
twenty-nine CARDINAL 15 18
Chattanooga GPE 31 32
the Pennsylvania Station FAC 48 51
a quarter to four CARDINAL 52 56
Baltimore GPE 64 65
Carolina GPE 82 83
Tennessee GPE 99 100
Chattanooga GPE 119 120
Chattanooga GPE 160 161
Chattanooga GPE 172 173


In [30]:
from collections import OrderedDict
places = [ent for ent in doc.ents if ent.label_ in ['GPE', 'LOC', 'FAC']]
unq_places = str({i.text for i in places})

In [32]:
unq_places

"{'Chattanooga', 'Baltimore', 'Tennessee', 'the Pennsylvania Station', 'Carolina'}"

In [38]:
top_songs.insert(5, 'Locations', 'places')

In [39]:
top_songs.at[0,'Locations']=unq_places
top_songs.head()

,Rank,Artist,Song,Year,Lyrics,Locations
0,1,Glenn Miller,Chattanooga Choo Choo,1941,"Chattanooga Choo Choo Pardon me, boy Is that t...","{'Chattanooga', 'Baltimore', 'Tennessee', 'the..."
1,2,Sammy Kaye,Daddy,1941,Daddy Ladada ladada da dada Ladada ladada da d...,places
2,3,Artie Shaw,Stardust,1941,Stardust And now the purple dusk of twilight t...,places
3,4,The Andrews Sisters,Boogie Woogie Bugle Boy,1941,Boogie Woogie Bugle Boy He was a famous trumpe...,places
4,5,Jimmy Dorsey,Amapola (Pretty Little Poppy),1941,words,places


In [40]:
for index, row in top_songs.iterrows():
    doc=nlp(top_songs.at[index,'Lyrics'])
    places = [ent for ent in doc.ents if ent.label_ in ['GPE', 'LOC', 'FAC']]
    unq_places = list({i.text for i in places})
    top_songs.at[index,'Locations'] = unq_places
    

C:\Users\josep\Anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Token indices sequence length is longer than the specified maximum sequence length for this model (634 > 512). Running this sequence through the model will result in indexing errors


In [41]:
top_songs.tail(50)

,Rank,Artist,Song,Year,Lyrics,Locations
6355,52,Sia,Elastic Heart,2015,"Elastic Heart [Intro] I won't give up, I won't...",[]
6356,53,Demi Lovato,Cool For The Summer,2015,Cool for the Summer [Verse 1] Tell me what you...,[]
6357,54,X Ambassadors,Renegades,2015,Renegades [Verse 1] Run away-ay with me (La la...,[]
6358,55,Usher feat. Juicy J,I Don’t Mind,2015,"I Don’t Mind [Produced by Cirkut, Dr. Luke, & ...",[]
6359,56,Ariana Grande and The Weeknd,Love Me Harder,2015,Love Me Harder [Verse 1: Ariana Grande] Tell m...,[]
6360,57,Taylor Swift,Wildest Dreams,2015,"Wildest Dreams [Verse 1] He said, ""Let's get o...",[]
6361,58,Sam Smith,Stay With Me,2015,"Stay With Me [Verse 1] Guess it's true, I'm no...",[]
6362,59,DJ Snake and Alunageorge,You Know You Like It,2015,You Know You Like It [Intro] Drives you insane...,[]
6363,60,Fall Out Boy,Uma Thurman,2015,Uma Thurman [Intro] I can move mountains I can...,[]
6364,61,Beyonce,7/11,2015,"7/11 [Pre-Chorus] Shoulders sideways, smack it...",[]


In [48]:
from pathlib import Path
filepath = Path('top_songs_locations.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
top_songs.to_csv(filepath)

In [51]:
good_list = top_songs.at[0,'Locations']

In [54]:
bad_list = top_songs.at[2,'Locations']

In [5]:
top_songs_copy = top_songs

NameError: name 'top_songs' is not defined

In [58]:
top_songs_copy.head()

,Rank,Artist,Song,Year,Lyrics,Locations
0,1,Glenn Miller,Chattanooga Choo Choo,1941,"Chattanooga Choo Choo Pardon me, boy Is that t...","[Baltimore, the Pennsylvania Station, Chattano..."
1,2,Sammy Kaye,Daddy,1941,Daddy Ladada ladada da dada Ladada ladada da d...,[Paris]
2,3,Artie Shaw,Stardust,1941,Stardust And now the purple dusk of twilight t...,[]
3,4,The Andrews Sisters,Boogie Woogie Bugle Boy,1941,Boogie Woogie Bugle Boy He was a famous trumpe...,[Chicago]
4,5,Jimmy Dorsey,Amapola (Pretty Little Poppy),1941,words,[]


In [4]:
rows_drop = []
for index, row in top_songs_copy.iterrows():
    if len(top_songs.at[index,'Locations'])==0:
        rows_drop.append(index)

NameError: name 'top_songs_copy' is not defined

In [65]:
geo_songs = top_songs_copy.drop(rows_drop)

In [67]:
geo_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 0 to 6402
Data columns (total 6 columns):
Rank         655 non-null int64
Artist       655 non-null object
Song         655 non-null object
Year         655 non-null int64
Lyrics       655 non-null object
Locations    655 non-null object
dtypes: int64(2), object(4)
memory usage: 35.8+ KB


In [2]:
from pathlib import Path
filepath = Path('top_songs_locations_only.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
geo_songs.to_csv(filepath)

NameError: name 'geo_songs' is not defined